**Coin-Row problem**: Given a row of `n` coins, with values `[c1, c2, ..., cn]`, the goal is to pick up the subset of coins with the largest combined value subject to the constraint that we canot pick any two adjacent coins.

Let `F(n)` be defines as the largest combined value given a row of `n` coins.

Using the dynamic programming approach, we consider two possible solutions:

(1) The solution subset includes the `nth` coin => it cannot contain the `(n-1)th` coin such that the remaining coins must be picked from the first `(n-2)` and the solution can be expressed as:

`F(n) = cn + F(n-2)`

(2) The solution subset does not contain the nth coin => the coins that can be picked must come from the first `(n-1)` coins, so the solution can be expressed as:

`F(n) = F(n-1)`


Then the final solution must be the larger of these two possibilities:

`F(n) = max(F(n-1), cn + F(n-2))`

which is a `recurrance relation` with base case `F(0) = 0, F(1) = c1`
